In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
def create_dataloaders(data_path, batch_size=32, train_split=0.8):
    data = pd.read_csv(data_path)
    features = data.iloc[:, :-1].values
    labels = data.iloc[:, -1].values

    class CustomDataset(Dataset):
        def __init__(self, features, labels):
            self.features = torch.tensor(features, dtype=torch.float32)
            self.labels = torch.tensor(labels, dtype=torch.long)

        def __len__(self):
            return len(self.features)

        def __getitem__(self, idx):
            return self.features[idx], self.labels[idx]

    # Shuffle and split data
    indices = np.arange(len(features))
    np.random.shuffle(indices)
    split_idx = int(len(indices) * train_split)

    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]

    train_dataset = CustomDataset(features[train_indices], labels[train_indices])
    test_dataset = CustomDataset(features[test_indices], labels[test_indices])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, features.shape[1], len(np.unique(labels))

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a sequence dimension
        h_0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)

        _, (hidden, _) = self.lstm(x, (h_0, c_0))
        out = self.fc(hidden[-1])
        return out

# Training and Evaluation Functions
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(100 * correct_train / total_train)

        # Validation
        model.eval()
        test_loss = 0
        correct_test = 0
        total_test = 0

        with torch.no_grad():
            for features, labels in test_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_test += (predicted == labels).sum().item()
                total_test += labels.size(0)

        test_losses.append(test_loss / len(test_loader))
        test_accuracies.append(100 * correct_test / total_test)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.2f}%, Test Acc: {test_accuracies[-1]:.2f}%")

    return train_losses, test_losses, train_accuracies, test_accuracies

# Plot Loss and Accuracy Graphs
def plot_metrics(train_losses, test_losses, train_accuracies, test_accuracies):
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(test_losses, label="Test Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss Curve")
    plt.legend()
    plt.grid()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label="Train Accuracy")
    plt.plot(test_accuracies, label="Test Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Accuracy Curve")
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()

# Main Execution
data_path = "data/gestures.csv"  # Change to your dataset path
batch_size = 32
hidden_size = 128
epochs = 120

train_loader, test_loader, input_size, num_classes = create_dataloaders(data_path, batch_size)

model = LSTMModel(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses, test_losses, train_accuracies, test_accuracies = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)
plot_metrics(train_losses, test_losses, train_accuracies, test_accuracies)

# Save the trained model
model_path = "lstm_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


Epoch [1/120], Train Loss: 5.9117, Test Loss: 5.6907, Train Acc: 22.97%, Test Acc: 72.41%
Epoch [2/120], Train Loss: 5.4536, Test Loss: 5.1503, Train Acc: 83.72%, Test Acc: 81.61%
Epoch [3/120], Train Loss: 4.7835, Test Loss: 4.3476, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [4/120], Train Loss: 3.7909, Test Loss: 3.2369, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [5/120], Train Loss: 2.5957, Test Loss: 2.0676, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [6/120], Train Loss: 1.6094, Test Loss: 1.2821, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [7/120], Train Loss: 1.0191, Test Loss: 0.8777, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [8/120], Train Loss: 0.7269, Test Loss: 0.6895, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [9/120], Train Loss: 0.5866, Test Loss: 0.5978, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [10/120], Train Loss: 0.5235, Test Loss: 0.5430, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [11/120], Train Loss: 0.4896, Test Loss: 0.5154, Train Acc: 84.59%, Test Acc: 81.61%
Epoch [1

: 